In [2]:
import pandas as pd
import numpy as np
import pymc as pm
from sklearn.model_selection import train_test_split

## Speed
Our response will be predicted get off time
    - Need to decide how we want to do this
    - Option 1 is amount of time to get to a certain distance (Say 1.5 yards)
    - Option 2 would be predicted yards given a cetain amount of time
For predictors:
    - Height/weight of the tackle
    - Quarter
    - Whether it's an obvious passing situation
    - Left vs right tackle?

## Power
Our response here is some variation of percentage of the distance to the QB given up once engaged with the defensive end
For predictors:
    - Height/weight of both the tackle and the end
    - Obvious passing situation
    - Time to throw for the QB
    - Quarter

In [3]:
model_data = pd.read_csv('https://media.githubusercontent.com/media/cnickol26/BigDataBowl2023/main/clustered.csv')

In [4]:
model_data.columns

Index(['uniqueplayId', 'QB_Flip', 'rushType', 'Power_prob', 'Speed_prob',
       'football_x', 'football_y', 'gameId', 'playId', 'frameId', 'time',
       'playDirection', 'event', 'ball_snap_frame', 'end_frame',
       'play_length_frames', 'Right_end', 'Left_end', 'QB_nflId',
       'QB_jerseyNumber', 'QB_team', 'QB_x', 'QB_y', 'QB_s', 'QB_a', 'QB_dis',
       'QB_o', 'QB_dir', 'QB_pff_role', 'QB_pff_positionLinedUp',
       'QB_pff_nflIdBlockedPlayer', 'QB_pff_blockType', 'QB_new_x', 'QB_new_y',
       'ball_nflId', 'ball_jerseyNumber', 'ball_team', 'ball_x', 'ball_y',
       'ball_s', 'ball_a', 'ball_dis', 'ball_o', 'ball_dir', 'ball_pff_role',
       'ball_pff_positionLinedUp', 'ball_pff_nflIdBlockedPlayer',
       'ball_pff_blockType', 'ball_new_x', 'ball_new_y', 'ball_Flip',
       'Tackle_nflId', 'Tackle_jerseyNumber', 'Tackle_team', 'Tackle_x',
       'Tackle_y', 'Tackle_s', 'Tackle_a', 'Tackle_dis', 'Tackle_o',
       'Tackle_dir', 'Tackle_pff_role', 'Tackle_pff_positionLined

In [5]:
model_data2 = model_data[['uniqueplayId','QB_Flip','rushType','Power_prob','Speed_prob',
            'football_x', 'football_y', 'gameId', 'playId', 'frameId', 'time',
       'event','play_length_frames','QB_s', 'QB_a', 'QB_dis',
       'QB_o', 'QB_dir', 'QB_pff_role', 'QB_pff_positionLinedUp','QB_new_x', 'QB_new_y',
            'ball_new_x', 'ball_new_y', 'ball_Flip','ball_snap_frame',
       'Tackle_nflId', 'Tackle_s', 'Tackle_a', 'Tackle_dis', 'Tackle_o',
       'Tackle_dir', 'Tackle_pff_role', 'Tackle_pff_positionLinedUp',
       'Tackle_pff_nflIdBlockedPlayer', 'Tackle_pff_blockType', 'Tackle_new_x',
       'Tackle_new_y', 'Tackle_Flip', 'End_nflId', 'End_s', 'End_a', 'End_dis', 'End_o',
       'End_dir', 'End_pff_role', 'End_pff_positionLinedUp', 'End_new_x',
       'End_new_y', 'End_Flip', 'tackle_end_dist', 'tackle_end_facing']]

In [6]:
players = pd.read_csv('https://media.githubusercontent.com/media/cnickol26/BigDataBowl2023/main/nfl-big-data-bowl-2023/players.csv')

In [7]:
players = players[['height', 'weight', 'nflId','displayName']]

In [8]:
model_data3 = model_data2.merge(players, left_on = 'Tackle_nflId', right_on = 'nflId', how = 'left')

In [9]:
model_data4 = model_data3.rename(columns = {'height':'tackle_height','weight':'tackle_weight','displayName':'tackle_name'}).drop('nflId', axis = 1)

In [10]:
model_data5 = model_data4.merge(players, left_on = 'End_nflId', right_on = 'nflId', how = 'left')

In [11]:
model_data6 = model_data5.rename(columns = {'height':'end_height','weight':'end_weight','displayName':'end_name'}).drop('nflId', axis = 1)

In [12]:
plays = pd.read_csv('https://media.githubusercontent.com/media/cnickol26/BigDataBowl2023/main/nfl-big-data-bowl-2023/plays.csv')

In [13]:
plays['obvious_pass'] = ((plays['down'] == 2) & (plays['yardsToGo'] > 10)) | ((plays['down'] == 3) & (plays['yardsToGo'] > 4)) | ((plays['down'] == 4) & (plays['yardsToGo'] > 2))

In [14]:
plays['obvious_pass'] = plays['obvious_pass'].astype(int)

In [15]:
plays = plays[['gameId','playId','quarter','obvious_pass']]

In [16]:
plays

,gameId,playId,quarter,obvious_pass
0,2021090900,97,1,0
1,2021090900,137,1,0
2,2021090900,187,1,0
3,2021090900,282,1,0
4,2021090900,349,1,1
...,...,...,...,...
8553,2021110100,4310,4,1
8554,2021110100,4363,4,0
8555,2021110100,4392,4,0
8556,2021110100,4411,4,1


In [17]:
model_data7 = model_data6.merge(plays, on = ['gameId','playId'], how = 'left')

In [18]:
model_data7['tackle_end_engaged'] = (model_data7['tackle_end_facing'] <= 40) & (model_data7['tackle_end_dist'] <= 1)

In [19]:
model_data7['tackle_end_engaged'] = model_data7['tackle_end_engaged'].astype(int)

In [20]:
model_data7['tackle_end_engaged'].value_counts()

0    201295
1     50412
Name: tackle_end_engaged, dtype: int64

In [21]:
def dist(x_1, y_1, x_2, y_2):
    return np.sqrt(np.sum([(x_1-x_2)**2, (y_1-y_2)**2], axis=0))

In [22]:
model_data7['tackle_qb_dist'] = dist(model_data7['Tackle_new_x'], model_data7['Tackle_new_y'], model_data7['QB_new_x'], model_data7['QB_new_y'])

In [23]:
model_data7['end_qb_dist'] = dist(model_data7['End_new_x'], model_data7['End_new_y'], model_data7['QB_new_x'], model_data7['QB_new_y'])

In [24]:
speed = model_data7[model_data7['rushType'] == 'Speed']
power = model_data7[model_data7['rushType'] == 'Power']

In [25]:
power_speed_perc = model_data7.groupby(['end_name', 'uniqueplayId', 'QB_Flip']).agg({
    'rushType':'first'
}).reset_index().groupby('end_name')['rushType'].agg(
    power_perc = lambda x: x[x == 'Power'].count()/x.count(),
    speed_perc = lambda x: x[x == 'Speed'].count()/x.count(),
    total = 'count'
).reset_index()

In [26]:
overall_perc_avs = model_data7.groupby(['uniqueplayId', 'QB_Flip']).agg({
    'rushType':'first'
}).reset_index()['rushType'].value_counts(normalize=True)
overall_perc_avs

Power    0.786609
Speed    0.213391
Name: rushType, dtype: float64

In [27]:
power_speed_perc['Av_Power'] = overall_perc_avs[0]
power_speed_perc['Av_Speed'] = overall_perc_avs[1]
power_speed_perc['num_plays'] = power_speed_perc['total'].sum()
power_speed_perc['end_power_perc'] = power_speed_perc['power_perc']*(power_speed_perc['total']/500) + power_speed_perc['Av_Power']*(1-power_speed_perc['total']/500)
power_speed_perc['end_speed_perc'] = power_speed_perc['speed_perc']*(power_speed_perc['total']/500) + power_speed_perc['Av_Speed']*(1-power_speed_perc['total']/500)

In [28]:
## Do we want to leave the tackles in the train set who don't have enough snaps to get test data? A couple need to be removed for sure since they are tight ends so I set keep if we have 5 instances
def split_data(data, tackles):
    train_x = pd.DataFrame()
    test_x = pd.DataFrame()
    for name in tackles:
        tackle = data[data['tackle_name'] == name]
        if len(tackle) > 4:
            if len(tackle) >= 25:
                x_train,x_test = train_test_split(tackle, test_size=0.2, random_state = 1029)
                train_x = pd.concat([train_x, x_train])
                test_x = pd.concat([test_x, x_test])
            else:
                train_x = pd.concat([train_x, tackle])
    return train_x, test_x

# Power Rushing Setup

In [29]:
subset = power[(power['tackle_end_engaged'] == 1)&(power['frameId'] >= power['ball_snap_frame']+5)]

In [30]:
subset2 = subset.groupby(['uniqueplayId','QB_Flip']).first().reset_index()

In [31]:
distance_when_engaged = subset2[['uniqueplayId','QB_Flip','tackle_qb_dist']]
distance_when_engaged

,uniqueplayId,QB_Flip,tackle_qb_dist
0,202109120163,0.0,3.927467
1,202109120163,1.0,5.853136
2,202109120676,0.0,4.894865
3,202109120676,1.0,3.900820
4,202109120776,0.0,4.369760
...,...,...,...
5434,20211025003660,1.0,5.355978
5435,20211025003904,0.0,3.735305
5436,20211025003904,1.0,4.439279
5437,20211025003926,0.0,4.778002


In [32]:
end_distance = power.groupby(['uniqueplayId','QB_Flip']).last().reset_index()

In [33]:
end_distance = end_distance[['uniqueplayId','QB_Flip','tackle_qb_dist']]
end_distance

,uniqueplayId,QB_Flip,tackle_qb_dist
0,202109120163,0.0,3.003265
1,202109120163,1.0,4.141618
2,202109120288,1.0,6.600947
3,202109120578,1.0,3.420322
4,202109120676,0.0,4.512704
...,...,...,...
7150,20211025003660,1.0,4.189141
7151,20211025003904,0.0,2.465218
7152,20211025003904,1.0,3.892364
7153,20211025003926,0.0,7.646829


In [34]:
end_distance = end_distance.rename(columns = {'tackle_qb_dist': 'ending_distance'})

In [35]:
distance = distance_when_engaged.merge(end_distance, on = ['uniqueplayId', 'QB_Flip'], how = 'left')
distance

,uniqueplayId,QB_Flip,tackle_qb_dist,ending_distance
0,202109120163,0.0,3.927467,3.003265
1,202109120163,1.0,5.853136,4.141618
2,202109120676,0.0,4.894865,4.512704
3,202109120676,1.0,3.900820,2.987072
4,202109120776,0.0,4.369760,3.070261
...,...,...,...,...
5434,20211025003660,1.0,5.355978,4.189141
5435,20211025003904,0.0,3.735305,2.465218
5436,20211025003904,1.0,4.439279,3.892364
5437,20211025003926,0.0,4.778002,7.646829


In [36]:
distance['pct_given_up'] = (distance['tackle_qb_dist'] - distance['ending_distance']) / distance['tackle_qb_dist']
distance

,uniqueplayId,QB_Flip,tackle_qb_dist,ending_distance,pct_given_up
0,202109120163,0.0,3.927467,3.003265,0.235318
1,202109120163,1.0,5.853136,4.141618,0.292410
2,202109120676,0.0,4.894865,4.512704,0.078074
3,202109120676,1.0,3.900820,2.987072,0.234245
4,202109120776,0.0,4.369760,3.070261,0.297385
...,...,...,...,...,...
5434,20211025003660,1.0,5.355978,4.189141,0.217857
5435,20211025003904,0.0,3.735305,2.465218,0.340022
5436,20211025003904,1.0,4.439279,3.892364,0.123199
5437,20211025003926,0.0,4.778002,7.646829,-0.600424


In [37]:
distance = distance.rename(columns = {'tackle_qb_dist' : 'starting_distance'})

In [38]:
model_data8 = power.merge(distance, on = ['uniqueplayId','QB_Flip'], how = 'left')
model_data8

,uniqueplayId,QB_Flip,rushType,Power_prob,Speed_prob,football_x,football_y,gameId,playId,frameId,...,end_weight,end_name,quarter,obvious_pass,tackle_end_engaged,tackle_qb_dist,end_qb_dist,starting_distance,ending_distance,pct_given_up
0,202109120163,0.0,Power,0.96468,0.03532,34.03,23.76,2021091201,63,6,...,242,Alex Highsmith,1,0,0,4.820290,7.786174,3.927467,3.003265,0.235318
1,202109120163,0.0,Power,0.96468,0.03532,34.03,23.76,2021091201,63,7,...,242,Alex Highsmith,1,0,0,4.816638,7.779460,3.927467,3.003265,0.235318
2,202109120163,0.0,Power,0.96468,0.03532,34.03,23.76,2021091201,63,8,...,242,Alex Highsmith,1,0,0,4.790918,7.744837,3.927467,3.003265,0.235318
3,202109120163,0.0,Power,0.96468,0.03532,34.03,23.76,2021091201,63,9,...,242,Alex Highsmith,1,0,0,4.766424,7.654672,3.927467,3.003265,0.235318
4,202109120163,0.0,Power,0.96468,0.03532,34.03,23.76,2021091201,63,10,...,242,Alex Highsmith,1,0,0,4.738143,7.531268,3.927467,3.003265,0.235318
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186954,20211021002175,1.0,Power,NaN,NaN,92.41,29.85,2021102100,2175,20,...,315,Jordan Elliott,3,0,0,4.995968,6.031923,NaN,NaN,NaN
186955,20211021002175,1.0,Power,NaN,NaN,92.41,29.85,2021102100,2175,21,...,315,Jordan Elliott,3,0,0,5.045909,6.051859,NaN,NaN,NaN
186956,20211021002175,1.0,Power,NaN,NaN,92.41,29.85,2021102100,2175,22,...,315,Jordan Elliott,3,0,0,5.093447,6.079753,NaN,NaN,NaN
186957,20211021002175,1.0,Power,NaN,NaN,92.41,29.85,2021102100,2175,23,...,315,Jordan Elliott,3,0,0,5.107886,6.099746,NaN,NaN,NaN


In [39]:
## Going to need to remove any power plays where this is an NA
model_data8 = model_data8.dropna(subset = ['pct_given_up'])
power = model_data8.merge(power_speed_perc[['end_name', 'end_power_perc', 'end_speed_perc']], how='left', on='end_name')

In [40]:
# Aggregate for grouped by to prepare data for modeling - one row per tackle/end instance
# The row should have the perctage of the distance given up once engaged, and multiple predictors
# The predictors are speed/power percent of the end, weight and height of end, one-hot offensive linesman, obvious passing situation, starting distance, length of the play, and maybe quarter
def full_agg(grouped_df):
    row = grouped_df[grouped_df['frameId']==grouped_df['ball_snap_frame']]
    return pd.DataFrame({
    'end_power_perc': row['end_power_perc'],
    'end_speed_perc':  row['end_speed_perc'],
    'end_height': row['end_height'],
    'end_weight': row['end_weight'],
    'tackle_name': row['tackle_name'],
    'obvious_pass': row['obvious_pass'],
    'quarter': row['quarter'],
    'pct_given_up': row['pct_given_up'],
    'starting_distance': row['starting_distance'],
    'play_length': row['play_length_frames']
})

power_data = power.groupby(['uniqueplayId', 'QB_Flip'], group_keys=True).apply(full_agg).reset_index().drop(['level_2'], axis=1)
power_data

,uniqueplayId,QB_Flip,end_power_perc,end_speed_perc,end_height,end_weight,tackle_name,obvious_pass,quarter,pct_given_up,starting_distance,play_length
0,202109120163,0.0,0.784777,0.215223,6-4,242,Dion Dawkins,0,1,0.235318,3.927467,22.0
1,202109120163,1.0,0.771070,0.228930,6-4,252,Daryl Williams,0,1,0.292410,5.853136,22.0
2,202109120676,0.0,0.794961,0.205039,6-2,270,Julien Davenport,0,1,0.078074,4.894865,21.0
3,202109120676,1.0,0.795338,0.204662,6-6,285,Braden Smith,0,1,0.234245,3.900820,21.0
4,202109120776,0.0,0.782241,0.217759,6-5,288,Taylor Lewan,1,1,0.297385,4.369760,27.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5434,20211025003660,1.0,0.794961,0.205039,6-2,270,Ryan Ramczyk,1,4,0.217857,5.355978,27.0
5435,20211025003904,0.0,0.791011,0.208989,6-6,265,Duane Brown,0,4,0.340022,3.735305,26.0
5436,20211025003904,1.0,0.792166,0.207834,6-4,287,Brandon Shell,0,4,0.123199,4.439279,26.0
5437,20211025003926,0.0,0.791011,0.208989,6-6,265,Duane Brown,0,4,-0.600424,4.778002,38.0


In [41]:
power_data['end_height'] = [int(j)*12+int(i) for j, i in power_data['end_height'].str.split('-')]
power_data['tackle_name'] = power_data['tackle_name'].str.replace(" ", "_")

power_dummies = pd.get_dummies(power_data, prefix="", prefix_sep='', columns=['tackle_name'])
power_dummies['tackle_name'] = power_data['tackle_name']
power_dummies

,uniqueplayId,QB_Flip,end_power_perc,end_speed_perc,end_height,end_weight,obvious_pass,quarter,pct_given_up,starting_distance,...,Trent_Williams,Tristan_Wirfs,Ty_Nsekhe,Ty_Sambrailo,Tyre_Phillips,Tyron_Smith,Yasir_Durant,Yodny_Cajuste,Yosuah_Nijman,tackle_name
0,202109120163,0.0,0.784777,0.215223,76,242,0,1,0.235318,3.927467,...,0,0,0,0,0,0,0,0,0,Dion_Dawkins
1,202109120163,1.0,0.771070,0.228930,76,252,0,1,0.292410,5.853136,...,0,0,0,0,0,0,0,0,0,Daryl_Williams
2,202109120676,0.0,0.794961,0.205039,74,270,0,1,0.078074,4.894865,...,0,0,0,0,0,0,0,0,0,Julien_Davenport
3,202109120676,1.0,0.795338,0.204662,78,285,0,1,0.234245,3.900820,...,0,0,0,0,0,0,0,0,0,Braden_Smith
4,202109120776,0.0,0.782241,0.217759,77,288,1,1,0.297385,4.369760,...,0,0,0,0,0,0,0,0,0,Taylor_Lewan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5434,20211025003660,1.0,0.794961,0.205039,74,270,1,4,0.217857,5.355978,...,0,0,0,0,0,0,0,0,0,Ryan_Ramczyk
5435,20211025003904,0.0,0.791011,0.208989,78,265,0,4,0.340022,3.735305,...,0,0,0,0,0,0,0,0,0,Duane_Brown
5436,20211025003904,1.0,0.792166,0.207834,76,287,0,4,0.123199,4.439279,...,0,0,0,0,0,0,0,0,0,Brandon_Shell
5437,20211025003926,0.0,0.791011,0.208989,78,265,0,4,-0.600424,4.778002,...,0,0,0,0,0,0,0,0,0,Duane_Brown


# Speed Rushing Setup

In [42]:
speed = speed.merge(power_speed_perc[['end_name', 'end_power_perc', 'end_speed_perc']], how='left', on='end_name')

In [43]:
speed['dist_moved'] = speed.groupby(['uniqueplayId','QB_Flip'])['Tackle_dis'].cumsum()

In [44]:
# Aggregate for grouped by to prepare data for modeling - one row per tackle/end instance
# The row should have the distance of the tackle in one second, and multiple predictors
# The predictors are speed/power percent of the end, weight and height of end, one-hot offensive linesman, obvious passing situation, and maybe quarter
def full_agg(grouped_df):
    row = grouped_df[grouped_df['frameId']==grouped_df['ball_snap_frame']+10]
    return pd.DataFrame({
    'one_sec_dist': row['dist_moved'],
    'end_power_perc': row['end_power_perc'],
    'end_speed_perc':  row['end_speed_perc'],
    'end_height': row['end_height'],
    'end_weight': row['end_weight'],
    'tackle_name': row['tackle_name'],
    'obvious_pass': row['obvious_pass'],
    'quarter': row['quarter']
})

speed_data = speed.groupby(['uniqueplayId', 'QB_Flip'], group_keys=True).apply(full_agg).reset_index().drop(['level_2'], axis=1)

In [45]:
speed_data['end_height'] = [int(j)*12+int(i) for j, i in speed_data['end_height'].str.split('-')]
speed_data['tackle_name'] = speed_data['tackle_name'].str.replace(" ", "_")
speed_data

,uniqueplayId,QB_Flip,one_sec_dist,end_power_perc,end_speed_perc,end_height,end_weight,tackle_name,obvious_pass,quarter
0,202109090097,0.0,2.51,0.786216,0.213784,77,255,Donovan_Smith,0,1
1,202109090097,1.0,1.46,0.784827,0.215173,75,245,Tristan_Wirfs,0,1
2,202109120288,0.0,1.36,0.777999,0.222001,75,275,Mekhi_Becton,0,1
3,202109120578,0.0,0.76,0.785472,0.214528,77,262,Laremy_Tunsil,0,1
4,202109190555,1.0,1.15,0.784743,0.215257,74,318,Jesse_Davis,0,1
...,...,...,...,...,...,...,...,...,...,...
1938,20211025003484,0.0,2.05,0.789572,0.210428,71,310,Terron_Armstead,0,4
1939,20211025003506,0.0,0.87,0.784082,0.215918,76,279,Terron_Armstead,1,4
1940,20211025003536,1.0,1.98,0.792718,0.207282,76,330,Ryan_Ramczyk,0,4
1941,20211025003660,0.0,1.95,0.787011,0.212989,76,260,Terron_Armstead,1,4


In [46]:
speed_dummies = pd.get_dummies(speed_data, prefix="", prefix_sep='', columns=['tackle_name'])
speed_dummies['tackle_name'] = speed_data['tackle_name']
speed_dummies

,uniqueplayId,QB_Flip,one_sec_dist,end_power_perc,end_speed_perc,end_height,end_weight,obvious_pass,quarter,Alejandro_Villanueva,...,Trent_Williams,Trenton_Brown,Tristan_Wirfs,Ty_Sambrailo,Tyre_Phillips,Tyron_Smith,Yasir_Durant,Yodny_Cajuste,Yosuah_Nijman,tackle_name
0,202109090097,0.0,2.51,0.786216,0.213784,77,255,0,1,0,...,0,0,0,0,0,0,0,0,0,Donovan_Smith
1,202109090097,1.0,1.46,0.784827,0.215173,75,245,0,1,0,...,0,0,1,0,0,0,0,0,0,Tristan_Wirfs
2,202109120288,0.0,1.36,0.777999,0.222001,75,275,0,1,0,...,0,0,0,0,0,0,0,0,0,Mekhi_Becton
3,202109120578,0.0,0.76,0.785472,0.214528,77,262,0,1,0,...,0,0,0,0,0,0,0,0,0,Laremy_Tunsil
4,202109190555,1.0,1.15,0.784743,0.215257,74,318,0,1,0,...,0,0,0,0,0,0,0,0,0,Jesse_Davis
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1938,20211025003484,0.0,2.05,0.789572,0.210428,71,310,0,4,0,...,0,0,0,0,0,0,0,0,0,Terron_Armstead
1939,20211025003506,0.0,0.87,0.784082,0.215918,76,279,1,4,0,...,0,0,0,0,0,0,0,0,0,Terron_Armstead
1940,20211025003536,1.0,1.98,0.792718,0.207282,76,330,0,4,0,...,0,0,0,0,0,0,0,0,0,Ryan_Ramczyk
1941,20211025003660,0.0,1.95,0.787011,0.212989,76,260,1,4,0,...,0,0,0,0,0,0,0,0,0,Terron_Armstead


# Power Modeling

In [47]:
power_tackles = power_data['tackle_name'].unique()

train_power, test_power = split_data(power_data, power_tackles)

In [48]:
# Amount of data removed by train test split
len(power_data) - len(train_power) - len(test_power)

28

In [49]:
power_data.head()

,uniqueplayId,QB_Flip,end_power_perc,end_speed_perc,end_height,end_weight,tackle_name,obvious_pass,quarter,pct_given_up,starting_distance,play_length
0,202109120163,0.0,0.784777,0.215223,76,242,Dion_Dawkins,0,1,0.235318,3.927467,22.0
1,202109120163,1.0,0.771070,0.228930,76,252,Daryl_Williams,0,1,0.292410,5.853136,22.0
2,202109120676,0.0,0.794961,0.205039,74,270,Julien_Davenport,0,1,0.078074,4.894865,21.0
3,202109120676,1.0,0.795338,0.204662,78,285,Braden_Smith,0,1,0.234245,3.900820,21.0
4,202109120776,0.0,0.782241,0.217759,77,288,Taylor_Lewan,1,1,0.297385,4.369760,27.0


In [58]:
players['displayName'] = players['displayName'].str.replace(" ", "_")

In [64]:
train_power2 = power_data.merge(players[['nflId', 'displayName']], left_on='tackle_name', right_on='displayName', how='left')

In [51]:
tackle_name = train_power['tackle_name'].values
end_power_perc =  train_power['end_power_perc'].values
end_height =  train_power['end_height'].values
end_weight =  train_power['end_weight'].values
obvious_pass =  train_power['obvious_pass'].values
quarter =  train_power['quarter'].values
play_length =  train_power['play_length'].values
unique_tackles = train_power['tackle_name'].nunique()

In [52]:
with pm.Model() as power_mod:

    # Priors
    mu_a = pm.Normal('mu_a', mu=0, sigma=10000)
    sigma_a = pm.Exponential("sigma_a", 0.5)

    # Random intercept
    a = pm.Normal('a', mu=mu_a, sigma=sigma_a, shape=train_power['tackle_name'].nunique())
    
    # Coefficients on predictors
    b1 = pm.Normal('b1', mu=0, sigma=100, shape=unique_tackles)
    b2 = pm.Normal('b2', mu=0, sigma=100, shape=unique_tackles)
    b3 = pm.Normal('b3', mu=0, sigma=100, shape=unique_tackles)
    b4 = pm.Normal('b4', mu=0, sigma=100, shape=unique_tackles)
    b5 = pm.Normal('b5', mu=0, sigma=100, shape=unique_tackles)
    b6 = pm.Normal('b6', mu=0, sigma=100, shape=unique_tackles)
    

    # Model error
    sigma_y = pm.Uniform('sigma_y', lower=0, upper=100)

    # Expected value
    y_hat = a[tackle_name] + b1*end_power_perc + b2*end_height +  b3*end_weight + b4*obvious_pass +  b5*quarter + b6*play_length

    # Data likelihood
    y_like = pm.Normal('y_like', mu=y_hat, sigma=sigma_y, observed=train_power['pct_given_up'])

pm.model_to_graphviz(power_mod)

ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: constant_folding
ERROR (pytensor.graph.rewriting.basic): node: InplaceDimShuffle{}(TensorConstant{(1,) of 102})
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/Users/joshgen/miniconda3/envs/pymc_env/lib/python3.11/site-packages/pytensor/graph/rewriting/basic.py", line 1933, in process_node
    replacements = node_rewriter.transform(fgraph, node)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/joshgen/miniconda3/envs/pymc_env/lib/python3.11/site-packages/pytensor/graph/rewriting/basic.py", line 1092, in transform
    return self.fn(fgraph, node)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/joshgen/miniconda3/envs/pymc_env/lib/python3.11/site-packages/pytensor/tensor/rewriting/basic.py", line 1141, in constant_folding
    thunk = node.op.make_thunk(node, storage_map, compute_map, no_recycling=[])
            ^^^^


You can find the C code in this temporary file: /var/folders/2k/10t0ng3n1pdd4bxjxwf5z3xw0000gn/T/pytensor_compilation_error_kcdkz539


ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: constant_folding
ERROR (pytensor.graph.rewriting.basic): node: InplaceDimShuffle{}(TensorConstant{(1,) of 102})
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/Users/joshgen/miniconda3/envs/pymc_env/lib/python3.11/site-packages/pytensor/graph/rewriting/basic.py", line 1933, in process_node
    replacements = node_rewriter.transform(fgraph, node)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/joshgen/miniconda3/envs/pymc_env/lib/python3.11/site-packages/pytensor/graph/rewriting/basic.py", line 1092, in transform
    return self.fn(fgraph, node)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/joshgen/miniconda3/envs/pymc_env/lib/python3.11/site-packages/pytensor/tensor/rewriting/basic.py", line 1141, in constant_folding
    thunk = node.op.make_thunk(node, storage_map, compute_map, no_recycling=[])
            ^^^^


You can find the C code in this temporary file: /var/folders/2k/10t0ng3n1pdd4bxjxwf5z3xw0000gn/T/pytensor_compilation_error_lhmt2dk0


ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: constant_folding
ERROR (pytensor.graph.rewriting.basic): node: InplaceDimShuffle{}(TensorConstant{(1,) of 102})
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/Users/joshgen/miniconda3/envs/pymc_env/lib/python3.11/site-packages/pytensor/graph/rewriting/basic.py", line 1933, in process_node
    replacements = node_rewriter.transform(fgraph, node)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/joshgen/miniconda3/envs/pymc_env/lib/python3.11/site-packages/pytensor/graph/rewriting/basic.py", line 1092, in transform
    return self.fn(fgraph, node)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/joshgen/miniconda3/envs/pymc_env/lib/python3.11/site-packages/pytensor/tensor/rewriting/basic.py", line 1141, in constant_folding
    thunk = node.op.make_thunk(node, storage_map, compute_map, no_recycling=[])
            ^^^^

...

ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: constant_folding
ERROR (pytensor.graph.rewriting.basic): node: InplaceDimShuffle{}(TensorConstant{(1,) of 102})
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/Users/joshgen/miniconda3/envs/pymc_env/lib/python3.11/site-packages/pytensor/graph/rewriting/basic.py", line 1933, in process_node
    replacements = node_rewriter.transform(fgraph, node)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/joshgen/miniconda3/envs/pymc_env/lib/python3.11/site-packages/pytensor/graph/rewriting/basic.py", line 1092, in transform
    return self.fn(fgraph, node)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/joshgen/miniconda3/envs/pymc_env/lib/python3.11/site-packages/pytensor/tensor/rewriting/basic.py", line 1141, in constant_folding
    thunk = node.op.make_thunk(node, storage_map, compute_map, no_recycling=[])
            ^^^^


You can find the C code in this temporary file: /var/folders/2k/10t0ng3n1pdd4bxjxwf5z3xw0000gn/T/pytensor_compilation_error_yi37290b


ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: constant_folding
ERROR (pytensor.graph.rewriting.basic): node: InplaceDimShuffle{}(TensorConstant{(1,) of 102})
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/Users/joshgen/miniconda3/envs/pymc_env/lib/python3.11/site-packages/pytensor/graph/rewriting/basic.py", line 1933, in process_node
    replacements = node_rewriter.transform(fgraph, node)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/joshgen/miniconda3/envs/pymc_env/lib/python3.11/site-packages/pytensor/graph/rewriting/basic.py", line 1092, in transform
    return self.fn(fgraph, node)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/joshgen/miniconda3/envs/pymc_env/lib/python3.11/site-packages/pytensor/tensor/rewriting/basic.py", line 1141, in constant_folding
    thunk = node.op.make_thunk(node, storage_map, compute_map, no_recycling=[])
            ^^^^


You can find the C code in this temporary file: /var/folders/2k/10t0ng3n1pdd4bxjxwf5z3xw0000gn/T/pytensor_compilation_error_kdw3tfbw


RecursionError: maximum recursion depth exceeded in comparison

# Speed Modeling